In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def LCT_parser(file_path):
    reports = []
    current_report = None
    inside_report = False
    next_line = None
    df = pd.DataFrame()
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip().startswith("Begin processing the"):
                event = line.strip().split()[8].rstrip(",")
            elif line.strip().startswith("-------- DQM/L1TMonitor LCT PRINTSTART --------"):
                inside_report = True
                current_report = {}
            elif line.strip().startswith("-------- DQM/L1TMonitor LCT PRINTEND --------"):
                inside_report = False
                reports.append(current_report)

            elif inside_report:
                line = line.strip()
                if line.startswith("Data LCT:") or line.startswith("Emulator LCT:"):
                    is_data = line.startswith("Data LCT:")
                    prev_line = line
                    next_line_t = True
                else:
                    next_line = line.strip()
                    if next_line_t:
                        info_parts = (prev_line + " " + next_line)
                        info_parts = info_parts.split()
                        data_lct_dict = pd.DataFrame({
                            'Source': [info_parts[0]],
                            'LCT#': [info_parts[4]],
                            'Valid': [info_parts[7]],
                            'BX': [info_parts[10]],
                            'Run-2 Pattern': [info_parts[14]],
                            'Run-3 Pattern': [info_parts[18]],
                            'Quality': [info_parts[21]],
                            'Bend': [info_parts[24]],
                            'Slope': [info_parts[27]],
                            'KeyHalfStrip': info_parts[30],
                            'KeyQuartStrip': info_parts[33],
                            'KeyEightStrip': info_parts[36],
                            'KeyWireGroup': info_parts[39],
                            'Type (SIM)': info_parts[43],
                            'MPC Link': info_parts[47],
                            'Event': [event]
                        })
                        df = df.append(data_lct_dict)
                        current_report.update(data_lct_dict)
                        next_line_t = False
    # df_list = []
    # for report in reports:
    #     df_list.append(pd.DataFrame([report]))

    # df = pd.concat(df_list, ignore_index=True)
    df = df.reset_index(drop=True, inplace=False)
    return df

# Usage
file_path = 'data/test.txt'
df = LCT_parser(file_path)


In [ ]:
def one_LCT_per_event_filter(df):
    for i in df['Event']:
        for j in df['LCT#'][df['Event'] == i]:
                if j != '#1:':
                    df = df[ df["Event"].str.contains(i)==False]
    # df = df.reset_index(drop=True, inplace=True)
    return df
df_one_LCT_per_event = one_LCT_per_event_filter(df).reset_index(drop=True, inplace=False)

In [ ]:
def plot_LCT_parameter(df, parameter):
    data = df[df['Source'] == 'Data']
    emulator = df[df['Source'] == 'Emulator']
    plt.figure(figsize=(10, 2))
    plt.scatter(data["Event"].astype(float), data[parameter].astype(float), c='red', s=3)
    plt.scatter(emulator["Event"].astype(float), emulator[parameter].astype(float), c='blue', s=3, alpha=0.5)
    plt.xlabel("Event")
    plt.ylabel(parameter) 
    j = 0
    k = 0
    for i in data['Event']:
        if len(emulator[emulator['Event'] == i]) == 1: #Check if for this event Emu LCT exists
            if data[data['Event'] == i][parameter].values[0] != emulator[emulator['Event'] == i][parameter].values[0]:
                j+=1
        k+=1
    plt.title("Mismatched LCT Data vs Emulator of " + parameter +"\nMismatch count: " + str(j) +"\nMismatch: " + str((j*100/k)) + "%" )
    plt.legend(['Data', 'Emulator'])
    plt.show()

j = int(0)
for i in df_one_LCT_per_event:
    if j > 3:
        plot_LCT_parameter(df_one_LCT_per_event, i)
    j+=1


In [ ]:
def plot_LCT_hist_parameter(df, parameter):
    data = df[df['Source'] == 'Data']
    emulator = df[df['Source'] == 'Emulator']
    plt.figure(figsize=(10, 2))
    plt.hist(data[parameter].astype(float), bins=100, color='red')
    plt.hist(emulator[parameter].astype(float), bins=100, color='blue', alpha=0.5)
    plt.xlabel(parameter)
    plt.ylabel("Count") 
    plt.title("Mismatched LCT Data vs Emulator of " + parameter)
    plt.legend(['Data', 'Emulator'])
    plt.show()
j = int(0)
for i in df:
    if j > 3:
        plot_LCT_hist_parameter(df_one_LCT_per_event, i)
    j+=1